Documentation: https://docs.unitycatalog.io/ai/quickstart/

In [0]:
%pip install -U -qqqq unitycatalog-ai[databricks] unitycatalog-langchain[databricks] mlflow databricks-langchain 
%restart_python

In [0]:
import mlflow

mlflow.langchain.autolog()

In [0]:
import sys, os, yaml
sys.path.append(os.path.abspath('..'))
from configs.project import ProjectConfig

with open("../configs/project.yml", "r") as file:
    data = yaml.safe_load(file)

projectConfig = ProjectConfig(**data)

In [0]:
uc_catalog = projectConfig.uc_catalog
uc_schema = projectConfig.uc_schema
llm_endpoint = projectConfig.llm_endpoint_names[0]
print(llm_endpoint)

## SQL Function

In [0]:
_ = spark.sql(f"DROP FUNCTION IF EXISTS {uc_catalog}.{uc_schema}.lookup_ticker_info")
_ = spark.sql(f"""
CREATE OR REPLACE FUNCTION {uc_catalog}.{uc_schema}.lookup_ticker_info(
  ticker_symbol STRING COMMENT 'Ticker symbol of the company whose info to look up'
)
RETURNS STRING
COMMENT 'Returns total current assets for a particular company, given the companys ticker symbol. The
ticker symbol can be used for other queries.'
RETURN SELECT CONCAT(
    'ticker symbol: ', TICKER, ', ',
    'total current assets: ', TOTAL_CURRENT_ASSETS
  )
  FROM {uc_catalog}.{uc_schema}.balance_sheet
  WHERE TICKER = ticker_symbol
  LIMIT 1;
""")

# Python Function

In [0]:
from databricks_langchain import ChatDatabricks
llm = ChatDatabricks(endpoint=llm_endpoint, temperature=0.1)

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

In [0]:
def add_numbers(number_1: float, number_2: float) -> float:
    """
    A function that accepts two floating point numbers, adds them,
    and returns the resulting sum as a float.

    Args:
        number_1 (float): The first of the two numbers to add.
        number_2 (float): The second of the two numbers to add.

    Returns:
        float: The sum of the two input numbers.
    """
    return number_1 + number_2

# Create the function in UC
function_info = client.create_python_function(
    func=add_numbers,
    catalog=uc_catalog,
    schema=uc_schema,
    replace=True,
)

In [0]:
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit

# Define the UC function to be used as a tool
func_name = f"{uc_catalog}.{uc_schema}.add_numbers"

# Create a toolkit with the UC function
toolkit = UCFunctionToolkit(function_names=[
  func_name,
  f"{uc_catalog}.{uc_schema}.lookup_ticker_info",
  ], client=client)

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from databricks_langchain import ChatDatabricks

# Initialize the LLM
llm = ChatDatabricks(endpoint=llm_endpoint, temperature = 0.0)

# Define the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools for computations if applicable."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create the agent with our tools
agent = create_tool_calling_agent(llm, toolkit.tools, prompt)

# Create the executor, adding our defined tools from the UCFunctionToolkit instance
agent_executor = AgentExecutor(agent=agent, tools=toolkit.tools, verbose=True)

# Run the agent with an input
# query = "What is the sum of 4321.9876 and 1234.5678?"
query = "What is the total current assets for Apple?"
agent_executor.invoke({"input": query})